In [ ]:
!pip install faiss-cpu tiktoken sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle
import tiktoken
import faiss
from sentence_transformers import SentenceTransformer

MODEL_NAME = "all-MiniLM-L6-v2"
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50
TOKENIZER_MODEL = "gpt-3.5-turbo"

DRIVE_PATH = "/content/drive/MyDrive/chunk_conv"
FILES = [
    os.path.join(DRIVE_PATH, "event.txt"),
]

In [ ]:
# -------------------------------
# Chunking Function (Token-based)
# -------------------------------
def chunk_by_tokens(text, chunk_size=512, overlap=100):
    enc = tiktoken.encoding_for_model(TOKENIZER_MODEL)
    tokens = enc.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = enc.decode(tokens[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

In [ ]:

# -------------------------------
# Load, Chunk & Embed
# -------------------------------
def main():
    print("[1] Loading text files...")
    all_chunks = []
    for file_path in FILES:
        with open(file_path, "r") as f:
            text = f.read()
            chunks = chunk_by_tokens(text, CHUNK_SIZE, CHUNK_OVERLAP)
            print(f"  - {file_path}: {len(chunks)} chunks")
            all_chunks.extend(chunks)

    print(f"[2] Total chunks: {len(all_chunks)}")

    print("[3] Embedding chunks...")
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(all_chunks)

    print("[4] Building FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print("[5] Saving index and chunks to Google Drive...")
    faiss.write_index(index, os.path.join(DRIVE_PATH, "rag_index.faiss"))
    with open(os.path.join(DRIVE_PATH, "rag_chunks.pkl"), "wb") as f:
        pickle.dump(all_chunks, f)

    print("[✓] Done! Files saved to Google Drive.")

if __name__ == "__main__":
    main()

[1] Loading text files...
  - /content/drive/MyDrive/chunk_conv/event.txt: 2 chunks
[2] Total chunks: 2
[3] Embedding chunks...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[4] Building FAISS index...
[5] Saving index and chunks to Google Drive...
[✓] Done! Files saved to Google Drive.
